In [1]:
from itertools import combinations

import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder

/Users/light/anaconda/lib/python3.5/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
date_cols = ['expiration_date', 'registration_init_time']

train_data = pd.read_hdf('../data/train_data.hdf', parse_dates=date_cols)
test_data = pd.read_hdf('../data/test_data.hdf', parse_dates=date_cols)

In [3]:
all_data = pd.concat([train_data, test_data])

In [4]:
df_test = test_data
df_history_test = train_data

df_final_trains = []
df_final_history_trains = []

n = len(test_data)
shift = int(0.05*len(train_data))

for i in range(2):
    m = -i*shift
    print (m, shift, i, -(n + i*shift), n)
    if m == 0:
        m = None
    df_final_trains.append(train_data[-(n + i*shift):m])
    df_final_history_trains.append(train_data[:-(n + i*shift)])
    print (len(df_final_trains[-1]))
    print (len(df_final_history_trains[-1]))

0 368870 0 -2556790 2556790
2556790
4820628
-368870 368870 1 -2925660 2556790
2556790
4451758


In [5]:
not_categorical_columns = [
    'target', 
    'song_length', 
    'registration_init_time', 
    'expiration_date', 
    'time', 
    'bd',
]
categorical_columns = all_data.columns.difference(not_categorical_columns)

orders = {}

for col in categorical_columns:
    orders[col] = 10 ** (int(np.log(all_data[col].max() + 1) / np.log(10)) + 1)

In [7]:
for col in orders:
    print (col, orders[col])

song_id 1000000
gender 10
source_type 100
lyricist 100000
msno 100000
source_screen_name 100
registered_via 10
city 100
language 100
composer 100000
source_system_tab 100
genre_ids 1000
artist_name 100000


In [38]:
def get_group(df, cols):
    
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
    return group
def mean(df_history, df, cols):
    
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)
    
    mean_map = df_history.groupby(group_history).target.mean()
    
    return group.map(mean_map).fillna(-1)
def regression(df_history, df, cols):
    
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)
    
    targets = {}
    times = {}
    for (y, t), u in zip(df_history[['target', 'time']].values, group_history):
        if u not in targets:
            targets[u] = [y]
            times[u] = [t]
        else:
            targets[u].append(y)
            times[u].append(t)
            
    linal_user = {}
    for u in times:
        if len(times[u]) > 1:
            A = np.vstack([times[u], np.ones(len(times[u]))]).T
            print (np.shape(A))
            linal_user[u] = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(targets[u])
    
    result = []
    
    for t, u in zip(df['time'], group):
        if u not in times:
            result.append(0.5)
        else:
            if len(times[u]) < 2:
                result.append(0.5)
            else:
                result.append(linal_user[u].dot([t, 1]))

    return result

In [9]:
def col_name(cols, func):
    return '_'.join(cols) + '_' + func.__name__

In [11]:
df = df_final_trains[0]
df_history = df_final_history_trains[0]

In [39]:
for num_col in [2]:
    for cols in combinations(categorical_columns, num_col):
        print (cols)
        regression(df, df_history,cols)
        break

('artist_name', 'city')
(9, 2)


LinAlgError: Singular matrix

In [17]:
print (list(cols))

['source_system_tab', 'source_type']


In [ ]:
get_group(df, list(cols))

In [36]:
group = get_group(df, cols)
group_history = get_group(df_history, cols)
for (y, t), u in zip(df_history[['target', 'time']].values, group_history):
    print (y, t, u)
    break

1.0 0.0 378400
